PRUEBA 1: EVALUACIÓN INTEGRAL DE TECNOLOGÍAS SOLAR
"PV vs CSP"

1) Selección de Países y Recolección de Datos:
Esta celda procesa los datos del recurso solar y condiciones meteorológicas de Chile, Sudáfrica Y China.

In [24]:
#Importar Librerías
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os

#direcciones
direccion_datos = '/home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos'
direccion_graficos_rs = '/home/nicole/proyecto/NicoleTorres/PRUEBA1/Graficos/Recurso Solar'
direccion_resultados = '/home/nicole/proyecto/NicoleTorres/PRUEBA1/Resultados'

Analisis de Recurso Solar

In [25]:
"""
Esta celda define las ubicaciones y los archivos de datos para Chile, China y Sudáfrica.
Y configura las funciones para analizar los datos y crear los gráficos de irradiación promedio mensual y horaria.
"""
#  Definir los pares de archivos por ubicación
ubicaciones = {
    'Sudafrica': {
        'lat': -29.19,
        'lon': 21.30,
        'elev': 0,  # Elevación por defecto
        'tz': 2,    # Zona horaria por defecto
        'archivos': ['1155731_-29.19_21.30_2018.csv', '1155731_-29.19_21.30_2019.csv']
    },
    'China': {
        'lat': 44.73,
        'lon': 87.66,
        'elev': 0,
        'tz': 8,
        'archivos': ['3480150_44.73_87.66_2018.csv', '3480150_44.73_87.66_2018.csv']
    },
    'Chile': {
        'lat': -23.84,
        'lon': -69.89,
        'elev': 735,  # Elevación real de Chile
        'tz': -4,
        'archivos': ['5815755_-23.84_-69.89_2018.csv', '5815755_-23.84_-69.89_2019.csv']
    }
}

def analizar_irradiacion_anual(df):
    """Analiza la irradiación anual por tipo"""
    resultados = {
        'GHI': {
            'promedio': float(df.select(pl.col('GHI').mean())[0,0]),
            'maximo': float(df.select(pl.col('GHI').max())[0,0]),
            'minimo': float(df.select(pl.col('GHI').min())[0,0]),
            'total': float(df.select(pl.col('GHI').sum())[0,0])
        },
        'DHI': {
            'promedio': float(df.select(pl.col('DHI').mean())[0,0]),
            'maximo': float(df.select(pl.col('DHI').max())[0,0]),
            'minimo': float(df.select(pl.col('DHI').min())[0,0]),
            'total': float(df.select(pl.col('DHI').sum())[0,0])
        },
        'DNI': {
            'promedio': float(df.select(pl.col('DNI').mean())[0,0]),
            'maximo': float(df.select(pl.col('DNI').max())[0,0]),
            'minimo': float(df.select(pl.col('DNI').min())[0,0]),
            'total': float(df.select(pl.col('DNI').sum())[0,0])
        }
    }
    return resultados

def analizar_irradiacion_mensual(df):
    """Analiza la irradiación mensual"""
    mensual = df.group_by('Month').agg(
        pl.col('GHI').mean().alias('GHI_promedio'),
        pl.col('DHI').mean().alias('DHI_promedio'),
        pl.col('DNI').mean().alias('DNI_promedio')
    ).sort('Month')
    return mensual

def crear_graficos(df, file_path, resultados_anuales, resultados_mensuales, ubicacion):
    """Crea los gráficos de análisis para una ubicación específica"""
    # Crear directorio de resultados si no existe
    if os.path.isdir(file_path):
        print(f"Graficos se guardaran en '{file_path}'.")
    else:
        os.makedirs(file_path, exist_ok=True)
        print(f"se creo la carpeta '{file_path}', donde se guardaran los graficos.") 
    
    
    # 1. Gráfico de irradiación diaria promedio por mes
    plt.figure(figsize=(12, 6))
    meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
             'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    
    plt.plot(meses, resultados_mensuales['GHI_promedio'], label='GHI', marker='o')
    plt.plot(meses, resultados_mensuales['DHI_promedio'], label='DHI', marker='s')
    plt.plot(meses, resultados_mensuales['DNI_promedio'], label='DNI', marker='^')
    
    plt.xlabel('Mes')
    plt.ylabel('Irradiancia Promedio (W/m²)')
    plt.title(f'Irradiancia Promedio Mensual en {ubicacion}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{file_path}/irradiacion_mensual_{ubicacion}.png')
    plt.close()
    
    # 2. Gráfico de distribución horaria
    horaria = df.group_by('Hour').agg(
        pl.col('GHI').mean().alias('GHI_promedio'),
        pl.col('DHI').mean().alias('DHI_promedio'),
        pl.col('DNI').mean().alias('DNI_promedio')
    ).sort('Hour')
    
    plt.figure(figsize=(12, 6))
    horas = range(24)
    
    plt.plot(horas, horaria['GHI_promedio'], label='GHI', marker='o')
    plt.plot(horas, horaria['DHI_promedio'], label='DHI', marker='s')
    plt.plot(horas, horaria['DNI_promedio'], label='DNI', marker='^')
    
    plt.xticks(range(0, 24, 1))
    plt.gca().set_xticklabels([f'{h:02d}:00' for h in range(24)])
    plt.xlabel('Hora del día (24h)')
    plt.ylabel('Irradiancia Promedio (W/m²)')
    plt.title(f'Distribución Horaria de la Irradiancia en {ubicacion}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{file_path}/distribucion_horaria_{ubicacion}.png')
    plt.close()



In [26]:
"""
Esta celda ejecuta el procesamiento de datos para cada ubicación.
""" 
# Procesar cada ubicación
for id_ubicacion, info in ubicaciones.items():
    print(f"\nProcesando ubicación {id_ubicacion} ({info['lat']}, {info['lon']})")
    
    # Lista para almacenar los DataFrames de cada año
    dfs = []
    
        # Procesar SOLO el primer archivo de la ubicación
    if not info['archivos']: # Verificar que la lista no esté vacía
        print(f"ADVERTENCIA: No se especificaron archivos para {id_ubicacion}. Saltando.")
        continue
        
    archivo_fuente = info['archivos'][0] # Tomar solo el primer archivo (ej., el de 2018)
    ruta_completa_fuente = os.path.join(direccion_datos, archivo_fuente)
    
    if not os.path.exists(ruta_completa_fuente):
        print(f"ADVERTENCIA: Archivo fuente {ruta_completa_fuente} no encontrado para {id_ubicacion}. Saltando.")
        continue
        
    # Leer el archivo CSV del año seleccionado, saltando las primeras 2 filas
    print(f"Leyendo archivo fuente: {ruta_completa_fuente}")
    try:
        # df_final ahora contendrá solo los datos del primer archivo
        df_final = pl.read_csv(ruta_completa_fuente, skip_rows=2) 
    except Exception as e:
        print(f"Error al leer {ruta_completa_fuente}: {e}. Saltando {id_ubicacion}.")
        continue
        
    # Seleccionar solo las columnas necesarias
    columnas_requeridas = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'GHI', 'DHI']
    if not all(col in df_final.columns for col in columnas_requeridas):
        print(f"ADVERTENCIA: Faltan columnas requeridas en {ruta_completa_fuente}. Columnas encontradas: {df_final.columns}. Saltando {id_ubicacion}.")
        continue
        
    df_final = df_final.select(columnas_requeridas)
        
    # Verificar número de registros (debería ser cercano a 8760)
    n_registros = len(df_final)
    if n_registros != 8760:
        print(f"ADVERTENCIA: El archivo {ruta_completa_fuente} tiene {n_registros} registros, se esperaban 8760.")
        
    # No necesitamos concatenar ni ordenar si solo usamos un archivo anual
    # df_final = pl.concat(dfs) <-- ELIMINADO
    # df_final = df_final.sort(['Year', 'Month', 'Day', 'Hour', 'Minute']) <-- ELIMINADO
    
    # Ordenar por fecha y hora
    df_final = df_final.sort(['Year', 'Month', 'Day', 'Hour', 'Minute'])
    
    # Realizar análisis de irradiación
    print(f"\nAnalizando irradiación para {id_ubicacion}...")
    resultados_anuales = analizar_irradiacion_anual(df_final)
    resultados_mensuales = analizar_irradiacion_mensual(df_final)
    
    # Crear gráficos
    print(f"Generando gráficos para {id_ubicacion}...")
    crear_graficos(df_final,direccion_graficos_rs, resultados_anuales, resultados_mensuales, id_ubicacion)
    
    # Imprimir resultados
    print(f"\nResultados del análisis para {id_ubicacion}:")
    for tipo in ['GHI', 'DHI', 'DNI']:
        print(f"\n{tipo}:")
        print(f"  Promedio: {resultados_anuales[tipo]['promedio']:.2f} W/m²")
        print(f"  Máximo: {resultados_anuales[tipo]['maximo']:.2f} W/m²")
        print(f"  Mínimo: {resultados_anuales[tipo]['minimo']:.2f} W/m²")
        print(f"  Total anual: {resultados_anuales[tipo]['total']/1000:.2f} kWh/m²")
    
    # Crear el archivo de salida con el formato PySAM
    nombre_archivo = f'{direccion_datos}/datos_{id_ubicacion}.csv'
    
    # Definir encabezados
    encabezado_metadata = 'Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation'
    encabezado_datos = df_final.columns # Columnas seleccionadas: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'GHI', 'DHI']

    # Escribir encabezados y datos
    with open(nombre_archivo, 'w') as f:
        # Línea 1: Encabezado de metadatos
        f.write(encabezado_metadata + '\n')

        # Línea 2: Valores de metadatos (con placeholders)
        source = 'ProcessedData' # O alguna otra fuente
        location_id = id_ubicacion
        city = id_ubicacion # Placeholder
        state = 'NA' # Placeholder
        country = id_ubicacion # Placeholder
        lat = info['lat']
        lon = info['lon']
        tz = info['tz']
        elev = info['elev']
        # Asegúrate de que todos los valores sean strings para unirlos con coma
        f.write(','.join(map(str, [source, location_id, city, state, country, lat, lon, tz, elev])) + '\n') 

        # Línea 3: Encabezado de datos horarios
        f.write(','.join(encabezado_datos) + '\n')

        # Líneas 4+: Datos horarios (sin encabezado del DataFrame)
        df_final.write_csv(f, include_header=False)
    
        print(f"\nLos datos han sido guardados en '{nombre_archivo}' con formato tipo antofagasta.csv")
    print(f"Los gráficos se han guardado en el directorio '{direccion_graficos_rs}'")

print("\nProcesamiento completado para todas las ubicaciones") 


Procesando ubicación Sudafrica (-29.19, 21.3)
Leyendo archivo fuente: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/1155731_-29.19_21.30_2018.csv

Analizando irradiación para Sudafrica...
Generando gráficos para Sudafrica...
Graficos se guardaran en '/home/nicole/proyecto/NicoleTorres/PRUEBA1/Graficos/Recurso Solar'.

Resultados del análisis para Sudafrica:

GHI:
  Promedio: 258.53 W/m²
  Máximo: 1182.00 W/m²
  Mínimo: 0.00 W/m²
  Total anual: 2264.71 kWh/m²

DHI:
  Promedio: 50.04 W/m²
  Máximo: 543.00 W/m²
  Mínimo: 0.00 W/m²
  Total anual: 438.33 kWh/m²

DNI:
  Promedio: 340.63 W/m²
  Máximo: 1103.00 W/m²
  Mínimo: 0.00 W/m²
  Total anual: 2983.94 kWh/m²

Los datos han sido guardados en '/home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/datos_Sudafrica.csv' con formato tipo antofagasta.csv
Los gráficos se han guardado en el directorio '/home/nicole/proyecto/NicoleTorres/PRUEBA1/Graficos/Recurso Solar'

Procesando ubicación China (44.73, 87.66)
Leyendo archivo fuente: /home/nicole

In [27]:
    # --- Bloque de verificación insertado ---
    if id_ubicacion == 'Chile':
        print(f"\\nVerificando estructura de {nombre_archivo} contra antofagasta.csv...")
        # Asegúrate que 'direccion_datos' esté definida correctamente antes de este punto.
        ruta_referencia = os.path.join(direccion_datos, 'antofagasta.csv') 
        try:
            with open(nombre_archivo, 'r') as f_gen, open(ruta_referencia, 'r') as f_ref:
                # Leer encabezados de metadatos (Línea 1)
                header_gen_meta = f_gen.readline().strip()
                header_ref_meta = f_ref.readline().strip()

                # Saltar línea de valores de metadatos (Línea 2)
                f_gen.readline()
                f_ref.readline()

                # Leer encabezados de datos (Línea 3)
                header_gen_data = f_gen.readline().strip()
                # El encabezado de referencia de datos se toma de las columnas usadas en el script
                # Asegúrate que 'columnas_requeridas' esté definida antes de este punto.
                header_ref_data_script = ','.join(columnas_requeridas) 

                # Comparar encabezados
                match_meta = header_gen_meta == header_ref_meta
                match_data = header_gen_data == header_ref_data_script

                print(f"  - Encabezado de metadatos (Línea 1) coincide con antofagasta.csv: {match_meta}")
                if not match_meta:
                    print(f"    > Generado: {header_gen_meta}")
                    print(f"    > Referencia (antofagasta.csv): {header_ref_meta}")

                print(f"  - Encabezado de datos (Línea 3) coincide con columnas del script: {match_data}")
                if not match_data:
                    print(f"    > Generado: {header_gen_data}")
                    print(f"    > Esperado (script): {header_ref_data_script}")

                if match_meta and match_data:
                    print("  => ¡La estructura de los encabezados generados es la esperada!")
                else:
                    print("  => ADVERTENCIA: La estructura de los encabezados generados NO coincide con la esperada.")

        except FileNotFoundError:
            print(f"  ERROR: No se pudo encontrar el archivo de referencia {ruta_referencia} o el archivo generado {nombre_archivo} para la verificación.")
        except Exception as e:
            print(f"  ERROR durante la verificación de estructura: {e}")
    # --- Fin del bloque insertado ---

\nVerificando estructura de /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/datos_Chile.csv contra antofagasta.csv...
  - Encabezado de metadatos (Línea 1) coincide con antofagasta.csv: True
  - Encabezado de datos (Línea 3) coincide con columnas del script: True
  => ¡La estructura de los encabezados generados es la esperada!


---------------------------------------------------------------------------------------------

2) Simulación de Plantas PV y CSP


Tecnología PV

In [22]:
#!/usr/bin/env python3
"""
Simulación de sistema fotovoltaico para múltiples países usando PySAM
Este script realiza simulaciones de sistemas fotovoltaicos para Chile, China y Sudáfrica
con diferentes capacidades de sistema
"""

import PySAM.Pvwattsv7 as pv
import PySAM.Lcoefcr as Lcoefcr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [23]:


# Configuración de parámetros del sistema
# ---------------------------------------
# Lista de países y sus archivos de recursos solares
paises = {
    "Chile": "datos_Chile.csv",
    "China": "datos_China.csv",
    "Sudafrica": "datos_Sudafrica.csv"
}

# Capacidades del sistema a simular (en kW)
capacidades = [500, 1000, 50000, 100000, 500000]

# Parámetros técnicos y económicos base
dc_ac_ratio = 1.2                 # Ratio DC/AC
tilt = 20                         # Inclinación de los paneles en grados
azimuth = 180                     # Azimut en grados (180 = orientación sur)
array_type = 1                    # 1 = montaje sobre techo con rack
gcr = 0.4                         # Ground Coverage Ratio
inv_eff = 96                      # Eficiencia del inversor (%)
losses = 14.0                     # Pérdidas del sistema (%)

# Parámetros económicos base
fixed_charge_rate = 0.07          # Tasa de carga fija (%)
fixed_operating_cost_base = 50_000     # Costo operativo fijo anual base ($/año)
variable_operating_cost = 0.01    # Costo operativo variable ($/kWh)

def calcular_costo_capital(capacidad_kw):
    """Calcula el costo de capital basado en la capacidad del sistema"""
    # Costo por kW disminuye con el tamaño del sistema
    costo_base_por_kw = 1000  # $/kW para sistemas pequeños
    factor_escala = 0.9  # Factor de reducción por escala
    costo_por_kw = costo_base_por_kw * (factor_escala ** np.log10(capacidad_kw/1000))
    return capacidad_kw * costo_por_kw

def calcular_costo_operativo_fijo(capacidad_kw):
    """Calcula el costo operativo fijo basado en la capacidad del sistema"""
    return fixed_operating_cost_base * (capacidad_kw / 1000) ** 0.8

def crear_directorio_resultados(pais, capacidad_kw):
    """Crea el directorio de resultados para un país y capacidad específica"""
    directorio = f"Resultados_{pais}/Capacidad_{capacidad_kw/1000:.0f}MW"
    if not os.path.exists(directorio):
        os.makedirs(directorio)
    return directorio

def simular_sistema_pv(pais, archivo_recurso, capacidad_kw):
    """Ejecuta la simulación del sistema fotovoltaico para un país y capacidad específica"""
    
    print(f"\n======== SIMULACIÓN PARA {pais} - {capacidad_kw/1000:.0f}MW ========")
    
    # Calcular costos específicos para esta capacidad
    capital_cost = calcular_costo_capital(capacidad_kw)
    fixed_operating_cost = calcular_costo_operativo_fijo(capacidad_kw)
    
    # Crear directorio de resultados
    directorio_resultados = crear_directorio_resultados(pais, capacidad_kw)
    
    # 1. Crear un modelo PVWatts
    print("Creando modelo PVWatts...")
    pv_model = pv.new()
    
    # 2. Asignar el archivo de recurso solar
    print(f"Usando archivo de recurso solar: {archivo_recurso}")
    pv_model.SolarResource.solar_resource_file = archivo_recurso
    
    # 3. Configurar los parámetros del sistema PV
    print("Configurando parámetros del sistema...")
    pv_model.SystemDesign.system_capacity = capacidad_kw
    pv_model.SystemDesign.dc_ac_ratio = dc_ac_ratio
    pv_model.SystemDesign.array_type = array_type
    pv_model.SystemDesign.azimuth = azimuth
    pv_model.SystemDesign.tilt = tilt
    pv_model.SystemDesign.gcr = gcr
    pv_model.SystemDesign.inv_eff = inv_eff
    pv_model.SystemDesign.losses = losses
    
    # 4. Ejecutar la simulación PVWatts
    print("Ejecutando simulación PVWatts...")
    pv_model.execute()
    
    # 5. Obtener los resultados
    print("Obteniendo resultados de la simulación...")
    annual_energy = pv_model.Outputs.annual_energy
    print(f"Generación anual de energía PV: {annual_energy:,.2f} kWh")
    
    # 6. Calcular el LCOE
    lcoe = calcular_lcoe(annual_energy, capital_cost, fixed_operating_cost)
    
    # 7. Guardar resultados en CSV
    guardar_resultados(annual_energy, lcoe, directorio_resultados, pais, capacidad_kw, capital_cost, fixed_operating_cost)
    
    # 8. Visualizar resultados mensuales
    visualizar_resultados_mensuales(pv_model, directorio_resultados, pais, capacidad_kw)
    
    # 9. Realizar análisis de sensibilidad
    analisis_sensibilidad_tilt(pv_model, directorio_resultados, pais, capacidad_kw)
    
    return pv_model, annual_energy, lcoe

def calcular_lcoe(annual_energy, capital_cost, fixed_operating_cost):
    """Calcula el LCOE (Levelized Cost of Energy)"""
    
    print("Calculando LCOE...")
    # Crear el modelo para cálculo de LCOE
    lcoe_model = Lcoefcr.new()

    # Asignar valores para el cálculo del LCOE
    lcoe_model.SimpleLCOE.annual_energy = annual_energy
    lcoe_model.SimpleLCOE.capital_cost = capital_cost
    lcoe_model.SimpleLCOE.fixed_charge_rate = fixed_charge_rate
    lcoe_model.SimpleLCOE.fixed_operating_cost = fixed_operating_cost
    lcoe_model.SimpleLCOE.variable_operating_cost = variable_operating_cost
    
    # Ejecutar el cálculo del LCOE
    lcoe_model.execute()
    
    # Obtener el LCOE calculado
    lcoe = lcoe_model.Outputs.lcoe_fcr
    print(f"LCOE: {lcoe:,.4f} $/kWh")
    
    return lcoe

def guardar_resultados(annual_energy, lcoe, directorio, pais, capacidad_kw, capital_cost, fixed_operating_cost):
    """Guarda los resultados en un archivo CSV"""
    
    print("Guardando resultados en CSV...")
    # Crear un DataFrame con los resultados
    df_results = pd.DataFrame({
        "Pais": [pais],
        "Capacidad_MW": [capacidad_kw/1000],
        "Annual_Energy_kWh": [annual_energy],
        "Capital_Cost_$": [capital_cost],
        "Fixed_Charge_Rate": [fixed_charge_rate],
        "Fixed_Operating_Cost_$/yr": [fixed_operating_cost],
        "Variable_Operating_Cost_$/kWh": [variable_operating_cost],
        "LCOE_$/kWh": [lcoe]
    })
    
    # Guardar en CSV
    ruta_archivo = os.path.join(directorio, f"resultados_{pais.lower()}_{capacidad_kw/1000:.0f}MW.csv")
    df_results.to_csv(ruta_archivo, index=False)
    print(f"Resultados guardados en {ruta_archivo}")

def visualizar_resultados_mensuales(pv_model, directorio, pais, capacidad_kw):
    """Visualiza la generación mensual de energía"""
    
    try:
        # Obtener generación mensual
        monthly_energy = pv_model.Outputs.monthly_energy
        
        # Crear gráfico de barras
        plt.figure(figsize=(10, 6))
        meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
        plt.bar(meses, monthly_energy)
        plt.title(f'Generación Mensual de Energía - {pais} - Sistema FV de {capacidad_kw/1000:.0f}MW')
        plt.xlabel('Mes')
        plt.ylabel('Energía (kWh)')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Guardar gráfico
        ruta_grafico = os.path.join(directorio, f'generacion_mensual_{pais.lower()}_{capacidad_kw/1000:.0f}MW.png')
        plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
        print(f"Gráfico de generación mensual guardado como '{ruta_grafico}'")
        
        plt.close()
    except Exception as e:
        print(f"No se pudo generar el gráfico: {e}")

def analisis_sensibilidad_tilt(pv_model, directorio, pais, capacidad_kw):
    """Realiza un análisis de sensibilidad variando la inclinación"""
    
    print(f"\nRealizando análisis de sensibilidad para la inclinación de los paneles en {pais}...")
    inclinaciones = [0, 10, 20, 30, 40]
    energias = []
    
    for tilt_value in inclinaciones:
        # Crear modelo para cada inclinación
        test_model = pv.new()
        test_model.SolarResource.solar_resource_file = pv_model.SolarResource.solar_resource_file
        
        # Configurar parámetros
        test_model.SystemDesign.system_capacity = capacidad_kw
        test_model.SystemDesign.dc_ac_ratio = dc_ac_ratio
        test_model.SystemDesign.array_type = array_type
        test_model.SystemDesign.azimuth = azimuth
        test_model.SystemDesign.tilt = tilt_value
        test_model.SystemDesign.gcr = gcr
        test_model.SystemDesign.inv_eff = inv_eff
        test_model.SystemDesign.losses = losses
        
        # Ejecutar simulación
        test_model.execute()
        
        # Obtener energía anual
        annual_energy = test_model.Outputs.annual_energy
        energias.append(annual_energy)
        print(f"  Inclinación: {tilt_value}°, Energía anual: {annual_energy:,.2f} kWh")
    
    # Crear gráfico
    plt.figure(figsize=(10, 6))
    plt.plot(inclinaciones, energias, marker='o', linestyle='-', linewidth=2)
    plt.title(f'Análisis de sensibilidad - Inclinación de paneles - {pais} - {capacidad_kw/1000:.0f}MW')
    plt.xlabel('Inclinación (grados)')
    plt.ylabel('Energía anual (kWh)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Guardar gráfico
    ruta_grafico = os.path.join(directorio, f'analisis_inclinacion_{pais.lower()}_{capacidad_kw/1000:.0f}MW.png')
    plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
    print(f"Gráfico de análisis de sensibilidad guardado como '{ruta_grafico}'")
    
    plt.close()

def comparar_capacidades(resultados):
    """Compara los resultados entre diferentes capacidades y genera gráficos comparativos"""
    
    print("\nGenerando comparación entre capacidades...")
    
    # Crear directorio para comparaciones
    directorio_comparacion = "Comparacion_Capacidades"
    if not os.path.exists(directorio_comparacion):
        os.makedirs(directorio_comparacion)
    
    # Definir colores para cada país
    colores = {'Chile': 'b', 'China': 'g', 'Sudafrica': 'r'}
    
    # Crear un gráfico que combine todos los países para energía vs capacidad
    plt.figure(figsize=(12, 8))
    
    for pais in paises.keys():
        # Preparar datos para comparación
        capacidades_mw = []
        energias = []
        
        for capacidad_kw in capacidades:
            if pais in resultados and capacidad_kw in resultados[pais]:
                capacidades_mw.append(capacidad_kw/1000)
                energias.append(resultados[pais][capacidad_kw]['annual_energy'])
        
        # Agregar curva al gráfico de energía
        plt.plot(capacidades_mw, energias, 
                color=colores[pais],
                marker='o', 
                linestyle='-', 
                linewidth=2,
                label=pais)
    
    # Configurar el gráfico de energía
    plt.title('Generación Anual de Energía vs Capacidad - Comparación entre Países')
    plt.xlabel('Capacidad (MW)')
    plt.ylabel('Energía anual (kWh)')
    plt.grid(True, alpha=0.3)
    plt.legend(title='País', loc='best')
    plt.tight_layout()
    
    # Guardar el gráfico de energía
    ruta_grafico = os.path.join(directorio_comparacion, 'energia_vs_capacidad_combinado.png')
    plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
    print(f"Gráfico de energía vs capacidad combinado guardado como '{ruta_grafico}'")
    
    plt.close()
    
    # Crear un gráfico que combine todos los países para LCOE vs capacidad
    plt.figure(figsize=(12, 8))
    
    for pais in paises.keys():
        # Preparar datos para comparación
        capacidades_mw = []
        lcoes = []
        
        for capacidad_kw in capacidades:
            if pais in resultados and capacidad_kw in resultados[pais]:
                capacidades_mw.append(capacidad_kw/1000)
                lcoes.append(resultados[pais][capacidad_kw]['lcoe'])
        
        # Agregar curva al gráfico de LCOE
        plt.plot(capacidades_mw, lcoes, 
                color=colores[pais],
                marker='o', 
                linestyle='-', 
                linewidth=2,
                label=pais)
    
    # Configurar el gráfico de LCOE
    plt.title('LCOE vs Capacidad - Comparación entre Países')
    plt.xlabel('Capacidad (MW)')
    plt.ylabel('LCOE ($/kWh)')
    plt.grid(True, alpha=0.3)
    plt.legend(title='País', loc='best')
    plt.tight_layout()
    
    # Guardar el gráfico de LCOE
    ruta_grafico = os.path.join(directorio_comparacion, 'lcoe_vs_capacidad_combinado.png')
    plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
    print(f"Gráfico de LCOE vs capacidad combinado guardado como '{ruta_grafico}'")
    
    plt.close()
    
    print(f"Gráficos de comparación guardados en el directorio '{directorio_comparacion}'")

def grafico_sensibilidad_combinado(resultados):
    """Genera un gráfico combinado de sensibilidad de inclinación para todas las capacidades de cada país"""
    
    print("\nGenerando gráficos combinados de sensibilidad de inclinación...")
    
    # Crear directorio para gráficos combinados
    directorio_combinado = "Graficos_Sensibilidad_Combinados"
    if not os.path.exists(directorio_combinado):
        os.makedirs(directorio_combinado)
    
    inclinaciones = [0, 10, 20, 30, 40]
    
    for pais in paises.keys():
        plt.figure(figsize=(12, 8))
        
        # Crear un gráfico para cada capacidad
        for capacidad_kw in capacidades:
            if pais in resultados and capacidad_kw in resultados[pais]:
                # Obtener datos de sensibilidad para esta capacidad
                energias = []
                for tilt_value in inclinaciones:
                    # Crear modelo para cada inclinación
                    test_model = pv.new()
                    test_model.SolarResource.solar_resource_file = paises[pais]
                    
                    # Configurar parámetros
                    test_model.SystemDesign.system_capacity = capacidad_kw
                    test_model.SystemDesign.dc_ac_ratio = dc_ac_ratio
                    test_model.SystemDesign.array_type = array_type
                    test_model.SystemDesign.azimuth = azimuth
                    test_model.SystemDesign.tilt = tilt_value
                    test_model.SystemDesign.gcr = gcr
                    test_model.SystemDesign.inv_eff = inv_eff
                    test_model.SystemDesign.losses = losses
                    
                    # Ejecutar simulación
                    test_model.execute()
                    
                    # Obtener energía anual
                    annual_energy = test_model.Outputs.annual_energy
                    energias.append(annual_energy)
                
                # Agregar curva al gráfico
                plt.plot(inclinaciones, energias, marker='o', linestyle='-', linewidth=2,
                        label=f'{capacidad_kw/1000:.0f}MW')
        
        # Configurar el gráfico
        plt.title(f'Análisis de Sensibilidad - Inclinación de Paneles - {pais}')
        plt.xlabel('Inclinación (grados)')
        plt.ylabel('Energía anual (kWh)')
        plt.grid(True, alpha=0.3)
        plt.legend(title='Capacidad del Sistema', loc='best')
        plt.tight_layout()
        
        # Guardar el gráfico
        ruta_grafico = os.path.join(directorio_combinado, f'sensibilidad_combinada_{pais.lower()}.png')
        plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
        print(f"Gráfico de sensibilidad combinada guardado como '{ruta_grafico}'")
        
        plt.close()

if __name__ == "__main__":
    print("======== SIMULACIÓN DE SISTEMAS FOTOVOLTAICOS PARA MÚLTIPLES PAÍSES Y CAPACIDADES ========")
    
    resultados = {}
    
    # Simular para cada país y cada capacidad
    for pais, archivo in paises.items():
        resultados[pais] = {}
        for capacidad_kw in capacidades:
            modelo, annual_energy, lcoe = simular_sistema_pv(pais, archivo, capacidad_kw)
            resultados[pais][capacidad_kw] = {
                'annual_energy': annual_energy,
                'lcoe': lcoe
            }
    
    # Generar comparación entre capacidades
    comparar_capacidades(resultados)
    
    # Generar gráficos combinados de sensibilidad
    grafico_sensibilidad_combinado(resultados)
    
    print("\n======== SIMULACIONES COMPLETADAS ========")
    print("Los resultados se han guardado en los respectivos directorios de cada país y capacidad")
    print("y se ha generado una comparación general en el directorio 'Comparacion_Capacidades'")
    print("Los gráficos combinados de sensibilidad se han guardado en el directorio 'Graficos_Sensibilidad_Combinados'") 

======== SIMULACIÓN DE SISTEMAS FOTOVOLTAICOS PARA MÚLTIPLES PAÍSES Y CAPACIDADES ========

======== SIMULACIÓN PARA Chile - 0MW ========
Creando modelo PVWatts...
Usando archivo de recurso solar: datos_Chile.csv
Configurando parámetros del sistema...
Ejecutando simulación PVWatts...


Exception: pvwattsv7 execution error.
	exec fail(pvwattsv7): could not open file for reading: datos_Chile.csv



Simulación CSP